In [1]:
import pandas as pd

In [2]:
data_coding = pd.read_csv('data/processed/coding_both.csv')
data_invoice = pd.read_csv('data/processed/invoices.csv')

# Data Preview

In [3]:
data_coding.head()


,ID,Supplier,Party,Total Spend,Expense Category,Transport,Miscellaneous,Catering,Accommodation,Expenses claimed by provider,...,Legal advice,Research,Polling,Focus groups,Ordinance survey data,Message testing,Archival research,Other forms of research,Social/Digital listening,Phone banking
0,0,Abi Bell,Liberal Democrats,1936.07,Transport,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Abigail Baker,Liberal Democrats,748.80,Transport,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Adam Prices,Plaid Cymru,275.85,Transport,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Adam Williams,Liberal Democrats,484.06,Transport,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Addison Lee,Conservative Party,22648.35,Transport,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
data_invoice.head()

,ECRef,RegulatedEntityName,ReportingPeriodName,RegulatedEntityType,TotalExpenditure,DateIncurred,ExpenseCategoryName,SupplierName,FullAddress,AmountInEngland,...,AmountInNorthernIreland,DateOfClaimForPayment,DatePaid,RedactedSupportingInvoiceId,RegulatedEntityId,SupplierId,JointCampaignName,UnregisteredCampaignerName,CampaigningName,IsOutsideSection75
0,SP0555253,Plaid Cymru,UK Parliamentary general election 12/12/2019,Political Party,200.40,03/01/2019,Unsolicited material to electors,Low Cost Printing,"Precision House, Pentrpoeth Rd, Swansea, SA6 6...",£0.00,...,£0.00,NaN,03/01/2019,74031,77,2276,NaN,NaN,NaN,False
1,SP0541026,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,60.00,01/04/2019,Advertising,Rayleigh Town Council,"The Pavilion, KGV Playing Field, Bull Lane, Ra...",£60.00,...,£0.00,01/04/2019,10/05/2019,0,90,68727,NaN,NaN,NaN,False
2,SP0541025,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,36.00,17/06/2019,Market research/canvassing,Rochford District Council,"Council Offices, South Street, ROCHFORD, SS4 1...",£36.00,...,£0.00,17/06/2019,04/07/2019,0,90,76767,NaN,NaN,NaN,False
3,SP0541024,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,9.99,07/01/2019,Advertising,RelyOn IT (UK),"1386 London Rd, Leigh-on-Sea, SS9 2UJ, United ...",£9.99,...,£0.00,07/01/2019,10/01/2019,0,90,76765,NaN,NaN,NaN,False
4,SP0541023,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,72.00,18/01/2019,Unsolicited material to electors,Off-Tech,"1 Brodie Road, ENFIELD, ZZ0 0ZZ, United Kingdom",£72.00,...,£0.00,18/01/2019,18/02/2019,0,90,76766,NaN,NaN,NaN,False


# Test

In [5]:
def get_accounted_output(name,accounted,unaccounted,overaccounted):
    acc_total = sum([accounted,unaccounted,overaccounted])
    return '{:<55s}'.format(name+':') \
            + '{:<40s}'.format('accounted = £'+'{0:.2f}'.format(accounted)+' ('+'{0:.1f}'.format((100*accounted/acc_total) if acc_total>0 else 0)+'%)') \
            + '{:<40s}'.format('| un-accounted = £'+'{0:.2f}'.format(unaccounted)+' ('+'{0:.1f}'.format((100*unaccounted/acc_total) if acc_total>0 else 0)+'%)') \
            + '{:<40s}'.format('| over-accounted = £'+'{0:.2f}'.format(overaccounted)+' ('+'{0:.1f}'.format((100*overaccounted/acc_total) if acc_total>0 else 0)+'%)')+'\n'


output=''
parties = data_coding['Party'].unique()
categories = data_coding['Expense Category'].unique()

total_accounted = 0
total_unaccounted = 0
total_overaccounted = 0

for party in parties:
    output += '\n####################################################################################\n  '+party+'\n####################################################################################\n\n'
    sub_party_coding = data_coding[ data_coding['Party']==party ]
    sub_party_invoice = data_invoice[ data_invoice['RegulatedEntityName']==party ]
    
    party_accounted = 0
    party_unaccounted = 0
    party_overaccounted = 0
    
    for category in categories:
        sub_cat_coding = sub_party_coding[ sub_party_coding['Expense Category']==category ]
        sub_cat_invoice = sub_party_invoice[ sub_party_invoice['ExpenseCategoryName']==category ]
        
        accounted = 0
        unaccounted = 0
        overaccounted = 0
        
        for idx, row in sub_cat_coding.iterrows():
            supplier = row['Supplier']
            spent = row['Total Spend']
            
            sub_pos_invoice = sub_cat_invoice[ sub_cat_invoice['SupplierName']==supplier ]
            filed = sub_pos_invoice['TotalExpenditure'].sum()

            accounted += min(spent,filed)
            unaccounted += max(spent-filed,0)
            overaccounted += max(filed-spent,0)
        
        party_accounted += accounted
        party_unaccounted += unaccounted
        party_overaccounted += overaccounted
    
        output += get_accounted_output(' Category '+category,accounted,unaccounted,overaccounted)
    output += '-'*170+'\n'
    output += get_accounted_output(' Total',party_accounted,party_unaccounted,party_overaccounted)
    
    total_accounted += party_accounted
    total_unaccounted += party_unaccounted
    total_overaccounted += party_overaccounted
    
print(get_accounted_output('Overall',total_accounted,total_unaccounted,total_overaccounted))
print(output)

Overall:                                               accounted = £33074672.83 (66.3%)        | un-accounted = £16486083.56 (33.1%)   | over-accounted = £299779.37 (0.6%)    


####################################################################################
  Liberal Democrats
####################################################################################

 Category Transport:                                   accounted = £265743.17 (97.6%)          | un-accounted = £522.05 (0.2%)         | over-accounted = £6110.62 (2.2%)      
 Category Advertising:                                 accounted = £2599322.26 (30.4%)         | un-accounted = £5961105.59 (69.6%)    | over-accounted = £0.00 (0.0%)         
 Category Unsolicited material to electors:            accounted = £3535320.69 (99.5%)         | un-accounted = £18432.91 (0.5%)       | over-accounted = £0.00 (0.0%)         
 Category Campaign broadcasts:                         accounted = £78305.40 (100.0%)          | un-acc